In [17]:
#  imports 
import numpy as np
import pandas as pd
from math import sqrt 
from pprint import pprint
from numpy import array

In [18]:
# load data
df=pd.read_csv('Cardiotocography_02_v10.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,LB,AC,FM,UC,ASTV,MSTV,ALTV,MLTV,DL,DS,...,Min,Max,Nmax,Nzeros,Mode,Mean,Median,Variance,Tendency,outlier
0,146.0,0.0,0.0,5.0,65.0,0.4,33.0,7.4,0.0,0.0,...,134.0,164.0,1.0,0.0,150.0,149.0,151.0,1.0,0.0,1
1,128.0,0.0,0.0,2.0,86.0,0.3,79.0,2.9,0.0,0.0,...,114.0,130.0,0.0,0.0,128.0,126.0,129.0,0.0,1.0,1
2,149.0,0.0,0.0,5.0,61.0,0.4,34.0,5.6,0.0,0.0,...,148.0,160.0,1.0,0.0,154.0,153.0,155.0,0.0,0.0,1
3,122.0,0.0,0.0,0.0,83.0,0.5,6.0,15.6,0.0,0.0,...,62.0,130.0,0.0,0.0,122.0,122.0,123.0,3.0,1.0,1
4,134.0,0.0,4.0,0.0,79.0,0.2,42.0,5.5,0.0,0.0,...,128.0,145.0,2.0,0.0,135.0,135.0,136.0,1.0,0.0,1


In [19]:
df.shape

(1688, 22)

In [20]:
# df to values
df = df.values

In [21]:
# Deep ExtraTrees
import argparse
import numpy as np
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/Cardiotocography.json")  
gc = GCForest(config)

In [22]:
# train test
from sklearn.cross_validation import train_test_split
y = df[:,21]
X = df[:,0:21]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [23]:
# count class
len(np.unique(y_train))

2

In [24]:
  # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
  

[ 2018-04-25 08:43:26,241][cascade_classifier.fit_transform] X_groups_train.shape=[(1181, 21)],y_train.shape=(1181,),X_groups_test.shape=[(507, 21)],y_test.shape=(507,)
[ 2018-04-25 08:43:26,244][cascade_classifier.fit_transform] group_dims=[21]
[ 2018-04-25 08:43:26,245][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 08:43:26,247][cascade_classifier.fit_transform] group_ends=[21]
[ 2018-04-25 08:43:26,249][cascade_classifier.fit_transform] X_train.shape=(1181, 21),X_test.shape=(507, 21)
[ 2018-04-25 08:43:26,251][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(1181, 21), X_cur_test.shape=(507, 21)
[ 2018-04-25 08:43:26,896][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.32%
[ 2018-04-25 08:43:27,687][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.31%
[ 2018-04-25 08:43:28,506][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_

In [25]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtratTrees = {:.6f} %".format(acc * 100))

[ 2018-04-25 08:43:34,242][cascade_classifier.transform] X_groups_test.shape=[(507, 21)]
[ 2018-04-25 08:43:34,244][cascade_classifier.transform] group_dims=[21]
[ 2018-04-25 08:43:34,246][cascade_classifier.transform] X_test.shape=(507, 21)
[ 2018-04-25 08:43:34,249][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(507, 21)


Test Accuracy of ExtratTrees = 97.633136 %


In [26]:
#  predict_proba + Data 
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-25 08:43:35,724][cascade_classifier.transform] X_groups_test.shape=[(507, 21)]
[ 2018-04-25 08:43:35,727][cascade_classifier.transform] group_dims=[21]
[ 2018-04-25 08:43:35,729][cascade_classifier.transform] X_test.shape=(507, 21)
[ 2018-04-25 08:43:35,731][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(507, 21)


In [27]:
# New data
X_train_new = X_train_enc
X_test_new  = X_test_enc

In [28]:
X_test_new.shape

(507, 23)

In [29]:
X_train_new.shape

(1181, 23)

In [30]:
# Cascade ExtraTrees
config = load_json("./examples/Cardiotocographygc.json")  
gc = GCForest(config)

X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-25 08:43:37,226][cascade_classifier.fit_transform] X_groups_train.shape=[(1181, 23)],y_train.shape=(1181,),X_groups_test.shape=[(507, 23)],y_test.shape=(507,)
[ 2018-04-25 08:43:37,228][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-04-25 08:43:37,229][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 08:43:37,231][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-04-25 08:43:37,233][cascade_classifier.fit_transform] X_train.shape=(1181, 23),X_test.shape=(507, 23)
[ 2018-04-25 08:43:37,235][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(1181, 23), X_cur_test.shape=(507, 23)
[ 2018-04-25 08:43:37,859][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.32%
[ 2018-04-25 08:43:38,649][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.31%
[ 2018-04-25 08:43:39,440][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_

In [31]:
# deep
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of Extratrees = {:.6f} %".format(acc * 100),i,'fois')

In [32]:
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    
    config = load_json("./examples/Cardiotocographygc.json")  
    gc = GCForest(config)
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()    

[ 2018-04-25 08:44:10,209][cascade_classifier.fit_transform] X_groups_train.shape=[(1181, 23)],y_train.shape=(1181,),X_groups_test.shape=[(507, 23)],y_test.shape=(507,)
[ 2018-04-25 08:44:10,211][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-04-25 08:44:10,212][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 08:44:10,214][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-04-25 08:44:10,215][cascade_classifier.fit_transform] X_train.shape=(1181, 23),X_test.shape=(507, 23)
[ 2018-04-25 08:44:10,217][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(1181, 23), X_cur_test.shape=(507, 23)
[ 2018-04-25 08:44:10,847][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.32%
[ 2018-04-25 08:44:11,763][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.31%
[ 2018-04-25 08:44:12,536][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_

[ 2018-04-25 08:44:44,365][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=98.31%
[ 2018-04-25 08:44:45,262][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=99.15%
[ 2018-04-25 08:44:46,073][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=98.31%
[ 2018-04-25 08:44:46,902][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=96.61%
[ 2018-04-25 08:44:47,745][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=99.15%
[ 2018-04-25 08:44:48,543][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=98.31%
[ 2018-04-25 08:44:49,371][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=99.15%
[ 2018-04-25 08:44:50,208][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=99.15%


('Test Accuracy of Extratrees = 97.830375 %', 0, 'fois')


[ 2018-04-25 08:45:04,793][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.32%
[ 2018-04-25 08:45:05,630][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.31%
[ 2018-04-25 08:45:06,453][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=99.15%
[ 2018-04-25 08:45:07,332][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=99.15%
[ 2018-04-25 08:45:08,095][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=96.61%
[ 2018-04-25 08:45:08,922][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.31%
[ 2018-04-25 08:45:09,767][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=99.15%
[ 2018-04-25 08:45:10,547][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.31%


[ 2018-04-25 08:45:43,074][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=99.15%
[ 2018-04-25 08:45:43,899][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=99.15%
[ 2018-04-25 08:45:44,704][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=99.15%
[ 2018-04-25 08:45:45,586][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_9.predict)=98.31%
[ 2018-04-25 08:45:45,734][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_cv.predict)=98.65%
[ 2018-04-25 08:45:45,736][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.test.predict)=97.83%
[ 2018-04-25 08:45:45,738][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=98.65%
[ 2018-04-25 08:45:45,740][cascade_classifier.calc_accuracy] Accuracy(layer_4 - test.classifier_average)=97.83%
[ 2018-04-25 08:45:45,742][

...

[ 2018-04-25 08:58:59,871][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=98.31%
[ 2018-04-25 08:59:00,722][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=100.00%
[ 2018-04-25 08:59:01,541][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=98.31%
[ 2018-04-25 08:59:02,359][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_9.predict)=98.31%
[ 2018-04-25 08:59:02,509][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_cv.predict)=98.48%
[ 2018-04-25 08:59:02,510][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.test.predict)=97.83%
[ 2018-04-25 08:59:02,512][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=98.48%
[ 2018-04-25 08:59:02,514][cascade_classifier.calc_accuracy] Accuracy(layer_4 - test.classifier_average)=97.83%
[ 2018-04-25 08:59:02,572]

('Test Accuracy of Extratrees = 97.830375 %', 18, 'fois')


[ 2018-04-25 08:59:25,705][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.32%
[ 2018-04-25 08:59:26,525][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.46%
[ 2018-04-25 08:59:27,304][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.31%
[ 2018-04-25 08:59:28,088][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.31%
[ 2018-04-25 08:59:28,864][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.46%
[ 2018-04-25 08:59:29,683][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.31%
[ 2018-04-25 08:59:30,462][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=99.15%
[ 2018-04-25 08:59:31,258][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=99.15%


[ 2018-04-25 09:00:02,618][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=98.31%
[ 2018-04-25 09:00:03,404][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=100.00%
[ 2018-04-25 09:00:04,199][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=98.31%
[ 2018-04-25 09:00:04,995][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_9.predict)=98.31%
[ 2018-04-25 09:00:05,144][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_cv.predict)=98.39%
[ 2018-04-25 09:00:05,145][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.test.predict)=97.83%
[ 2018-04-25 09:00:05,147][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=98.39%
[ 2018-04-25 09:00:05,149][cascade_classifier.calc_accuracy] Accuracy(layer_4 - test.classifier_average)=97.83%
[ 2018-04-25 09:00:05,151]

('Test Accuracy of Extratrees = 97.830375 %', 19, 'fois')


# ('Test Accuracy of Extratrees = 97.830375 %', 0, 'fois')